In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
driver_locations = "/content/gdrive/MyDrive/week_8_data/driver_locations_during_request.csv"
data = "/content/gdrive/MyDrive/week_8_data/data_cleaning.csv"

Mounted at /content/gdrive


In [2]:
import pandas as pd
df= pd.read_csv(data)

In [3]:
df.head()

,Unnamed: 0,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,order_id,driver_id,driver_action,lat,lng,start_hour,end_hour,trip_distance_km,trip_time,rainy,weekend
0,10,1259098,"6.444648,3.489271","6.586237700000001,3.2173232",2021-11-13 13:47:32,2021-11-13 16:02:43,392005,245597,accepted,6.549147,3.392184,13.0,16,33.919188,0 days 02:15:11,False,True
1,11,1259098,"6.444648,3.489271","6.586237700000001,3.2173232",2021-11-13 13:47:32,2021-11-13 16:02:43,392005,243172,rejected,6.570408,3.396597,13.0,16,33.919188,0 days 02:15:11,False,True
2,12,1259098,"6.444648,3.489271","6.586237700000001,3.2173232",2021-11-13 13:47:32,2021-11-13 16:02:43,392005,243392,rejected,6.566144,3.366483,13.0,16,33.919188,0 days 02:15:11,False,True
3,13,1259098,"6.444648,3.489271","6.586237700000001,3.2173232",2021-11-13 13:47:32,2021-11-13 16:02:43,392005,243281,rejected,6.576791,3.389592,13.0,16,33.919188,0 days 02:15:11,False,True
4,14,1259098,"6.444648,3.489271","6.586237700000001,3.2173232",2021-11-13 13:47:32,2021-11-13 16:02:43,392005,244078,rejected,6.577215,3.371172,13.0,16,33.919188,0 days 02:15:11,False,True


# Categorize trip_distance_km
Define categories for trip_distance_km based on some threshold values. Here, we'll categorize it into 'low', 'medium', and 'high'.

In [4]:
# Define thresholds for distance categories (example thresholds)
low_threshold = 5.0
high_threshold = 100.0

# Categorize trip_distance_km
df['distance_category'] = pd.cut(df['trip_distance_km'], bins=[-float('inf'), low_threshold, high_threshold, float('inf')],
                                 labels=['low distance', 'medium distance', 'high distance'])

In [5]:
!pip install causalinference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 1.8 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from causalinference import CausalModel

In [7]:
# Categorize trip_distance_km into 'low', 'medium', 'high' based on quartiles
low_threshold = df['trip_distance_km'].quantile(0.25)
high_threshold = df['trip_distance_km'].quantile(0.75)

df['distance_category'] = pd.cut(df['trip_distance_km'], bins=[-float('inf'), low_threshold, high_threshold, float('inf')],
                                 labels=['low distance', 'medium distance', 'high distance'])

# Print first few rows to verify
print(df.head())


   Unnamed: 0  Trip ID        Trip Origin             Trip Destination  \
0          10  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
1          11  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
2          12  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
3          13  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
4          14  1259098  6.444648,3.489271  6.586237700000001,3.2173232   

       Trip Start Time        Trip End Time  order_id  driver_id  \
0  2021-11-13 13:47:32  2021-11-13 16:02:43    392005     245597   
1  2021-11-13 13:47:32  2021-11-13 16:02:43    392005     243172   
2  2021-11-13 13:47:32  2021-11-13 16:02:43    392005     243392   
3  2021-11-13 13:47:32  2021-11-13 16:02:43    392005     243281   
4  2021-11-13 13:47:32  2021-11-13 16:02:43    392005     244078   

  driver_action       lat       lng  start_hour  end_hour  trip_distance_km  \
0      accepted  6.549147  3.392184        13.0        16         3

In [13]:
# Convert Trip Start Time and Trip End Time to datetime objects
df['Trip Start Time'] = pd.to_datetime(df['Trip Start Time'])
df['Trip End Time'] = pd.to_datetime(df['Trip End Time'])

# Define a function to categorize time periods
def categorize_time_period(start_time, end_time):
    hour = start_time.hour
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'late-afternoon'
    else:
        return 'night'

# Apply categorization to create 'time_period' column
df['time_period'] = df.apply(lambda row: categorize_time_period(row['Trip Start Time'], row['Trip End Time']), axis=1)

# Define function to categorize into working-hour, free-time, peak-hour
def categorize_time_slot(start_hour):
    if 9 <= start_hour < 17:  # Working-hour: 9am to 5pm
        return 'working-hour'
    elif 17 <= start_hour < 21:  # Peak-hour: 5pm to 9pm
        return 'peak-hour'
    else:  # Free-time: 9pm to 9am
        return 'free-time'

# Apply categorization to create 'time_slot' column
df['time_slot'] = df['start_hour'].apply(categorize_time_slot)

# Print first few rows to verify
print(df.head())

   Unnamed: 0  Trip ID        Trip Origin             Trip Destination  \
0          10  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
1          11  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
2          12  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
3          13  1259098  6.444648,3.489271  6.586237700000001,3.2173232   
4          14  1259098  6.444648,3.489271  6.586237700000001,3.2173232   

      Trip Start Time       Trip End Time  order_id  driver_id driver_action  \
0 2021-11-13 13:47:32 2021-11-13 16:02:43    392005     245597      accepted   
1 2021-11-13 13:47:32 2021-11-13 16:02:43    392005     243172      rejected   
2 2021-11-13 13:47:32 2021-11-13 16:02:43    392005     243392      rejected   
3 2021-11-13 13:47:32 2021-11-13 16:02:43    392005     243281      rejected   
4 2021-11-13 13:47:32 2021-11-13 16:02:43    392005     244078      rejected   

        lat  ...  trip_distance_km        trip_time  rainy  weekend  \
0  

#Causal Inference
we will perform Causal Iinference using **causalinference**

In [8]:

# Convert categorical variables to integers for causalinference
df['treated'] = np.where(df['driver_action'] == 'accepted', 1, 0)
df['rainy_confounders'] = np.where(df['rainy'] == 'True', 1, 0)
df['weekend_confounders'] = np.where(df['weekend'] == 'True', 1, 0)

# Define the CausalModel
causal_model = CausalModel(
    Y=df['trip_distance_km'].values,  # Outcome variable
    D=df['treated'].values,  # Treatment variable
    X=df[['rainy_confounders', 'weekend_confounders']].values  # Potential confounders
)

# Estimate treatment effect using propensity score matching
causal_model.est_via_matching(matches=1, bias_adj=True)

# Print the average treatment effect
print(causal_model.estimates)

# Plot the results
# Visualizing the outcomes
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df[df['Treatment'] == 0]['Outcome'], alpha=0.5, label='Control', color='blue')
plt.hist(df[df['Treatment'] == 1]['Outcome'], alpha=0.5, label='Treated', color='red')
plt.title('Distribution of Outcomes')
plt.xlabel('Outcome')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 2, 2)
treated_mean = df[df['Treatment'] == 1]['Outcome'].mean()
control_mean = df[df['Treatment'] == 0]['Outcome'].mean()
plt.bar(['Control', 'Treated'], [control_mean, treated_mean], color=['blue', 'red'])
plt.title('Average Outcome by Group')
plt.ylabel('Average Outcome')

plt.tight_layout()
plt.show()


/usr/local/lib/python3.10/dist-packages/causalinference/core/summary.py:110: RuntimeWarning: invalid value encountered in divide
  return (mean_t-mean_c) / np.sqrt((sd_c**2+sd_t**2)/2)


In [14]:
# Convert categorical variables to integers for causalinference
df['treated'] = np.where(df['driver_action'] == 'accepted', 1, 0)
df['rainy_confounders'] = np.where(df['rainy'] == 'True', 1, 0)
df['weekend_confounders'] = np.where(df['weekend'] == 'True', 1, 0)

# Define the CausalModel
causal_model = CausalModel(
    Y=df['time_period'].values,  # Outcome variable
    D=df['treated'].values,  # Treatment variable
    X=df[['rainy_confounders', 'weekend_confounders']].values  # Potential confounders
)

# Estimate treatment effect using propensity score matching
causal_model.est_via_ols()
 #causal_model.est_via_matching(matches=1, bias_adj=True)

# Print the average treatment effect
print(causal_model.estimates)

# Plot the results
# Visualizing the outcomes
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df[df['Treatment'] == 0]['Outcome'], alpha=0.5, label='Control', color='blue')
plt.hist(df[df['Treatment'] == 1]['Outcome'], alpha=0.5, label='Treated', color='red')
plt.title('Distribution of Outcomes')
plt.xlabel('Outcome')
plt.ylabel('Frequency')
plt.legend()

plt.subplot(1, 2, 2)
treated_mean = df[df['Treatment'] == 1]['Outcome'].mean()
control_mean = df[df['Treatment'] == 0]['Outcome'].mean()
plt.bar(['Control', 'Treated'], [control_mean, treated_mean], color=['blue', 'red'])
plt.title('Average Outcome by Group')
plt.ylabel('Average Outcome')

plt.tight_layout()
plt.show()


TypeError: ufunc 'divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

# CausalGraphicalModels

In [ ]:
from causalgraphicalmodels import CausalGraphicalModel

In [ ]:
traffic_condition = CausalGraphicalModel(
    nodes=["time_period", "time_slot", "morning", "afternoon", "late-afternoon","night", "working-hour", "pick-hour", "free-time"],
    edges=[
        ("time_period", "morning"),
        ("time_period", "afternoon"),
        ("time_period", "late-afternoon"),
        ("time_period", "night"),
        ("morning", "time_slot"),
        ("afternoon", "time_slot"),
        ("late-afternoon", "time_slot"),
        ("night", "time_slot"),
        ("time_slot", "working-hour"),
        ("time_slot", "pick-hour"),
        ("time_slot", "free-time")
    ]
)

# draw return a graphviz `dot` object, which jupyter can render
traffic_condition.draw()